Table: `Ads`


| Column Name   | Type    |
|---------------|---------|
| ad_id         | int     |
| user_id       | int     |
| action        | enum    |

(ad_id, user_id) is the primary key for this table.

Each row of this table contains the ID of an Ad, the ID of a user and the action taken by this user regarding this Ad.

The action column is an ENUM type of ('Clicked', 'Viewed', 'Ignored').
 

A company is running Ads and wants to calculate the performance of each Ad.

Performance of the Ad is measured using Click-Through Rate (CTR) where:

![](sql1.png)

Write an SQL query to find the ctr of each Ad.

Round ctr to 2 decimal points. Order the result table by ctr in descending order and by ad_id in ascending order in case of a tie.

The query result format is in the following example:

Ads table:

| ad_id | user_id | action  |
|-------|---------|---------|
| 1     | 1       | Clicked |
| 2     | 2       | Clicked |
| 3     | 3       | Viewed  |
| 5     | 5       | Ignored |
| 1     | 7       | Ignored |
| 2     | 7       | Viewed  |
| 3     | 5       | Clicked |
| 1     | 4       | Viewed  |
| 2     | 11      | Viewed  |
| 1     | 2       | Clicked |

Result table:

| ad_id | ctr   |
|-------|-------|
| 1     | 66.67 |
| 3     | 50.00 |
| 2     | 33.33 |
| 5     | 0.00  |

for ad_id = 1, ctr = (2/(2|1)) * 100 = 66.67

for ad_id = 2, ctr = (1/(1|2)) * 100 = 33.33

for ad_id = 3, ctr = (1/(1|1)) * 100 = 50.00

for ad_id = 5, ctr = 0.00, Note that ad_id = 5 has no clicks or views.

Note that we don't care about Ignored Ads.

Result table is ordered by the ctr. in case of a tie we order them by ad_id

In [1]:
# Create connection:
import mysql.connector
import pandas as pd
def query(query):
        mySql_connector = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="leedcode_test"
        )
        data= pd.read_sql_query(query , mySql_connector)
        return data


In [2]:
query("""
SELECT * FROM Ads;
""")
    

,ad_id,user_id,action
0,1,1,Clicked
1,1,2,Clicked
2,1,4,Viewed
3,1,7,Ignored
4,2,2,Clicked
5,2,7,Viewed
6,2,11,Viewed
7,3,3,Viewed
8,3,5,Clicked
9,5,5,Ignored


In [3]:
query('''
 SELECT ad_id,
          SUM(CASE
                  WHEN action ='Viewed' THEN 1
                  ELSE 0
              END) views,
          SUM(CASE
                  WHEN action = 'Clicked' THEN 1
                  ELSE 0
              END) Click
   FROM Ads
   GROUP BY ad_id
''')

,ad_id,views,Click
0,1,1.0,2.0
1,2,2.0,1.0
2,3,1.0,1.0
3,5,0.0,0.0


In [4]:
query('''

# Time:  O(nlogn)
# Space: O(n)

SELECT ad_id,
         #write our formula
       CASE
           WHEN clicks + views = 0 THEN 0 # case 1
           ELSE ROUND(100 * clicks / (clicks + views), 2)  # case 2
       END ctr
FROM
  (SELECT ad_id,
          SUM(CASE
                  WHEN action ='Viewed' THEN 1
                  ELSE 0
              END) views,
          SUM(CASE
                  WHEN action = 'Clicked' THEN 1
                  ELSE 0
              END) clicks
   FROM Ads
   GROUP BY ad_id) a
ORDER BY ctr DESC ,  ad_id ASC;


''')

,ad_id,ctr
0,1,66.67
1,3,50.00
2,2,33.33
3,5,0.00
